In [ ]:
import random
import pandas as pd

In [ ]:
YDS = pd.read_csv('C:/Users/김태성/Desktop/파이썬용CSV/팀빌딩.csv')

# pandas에서 각 row마다 개별 list로 바꾸기
A = [];B = [];C = [];D = [];E = [];F = []
for x in range(len(YDS)):
    if YDS.loc[x][3] == '0':
        F.append(YDS.loc[x].tolist()) # 설문 미제출자는 1,2,3지망 모두 '0'처리
    elif (YDS.loc[x][3] == YDS.loc[x][4]) & (YDS.loc[x][4] == YDS.loc[x][5]) & (YDS.loc[x][3] != '0'):
        A.append(YDS.loc[x].tolist()) # 1,2,3지망 모두 같음
    elif (YDS.loc[x][3] == YDS.loc[x][4]) & (YDS.loc[x][3] != '0'):
        B.append(YDS.loc[x].tolist())# 1,2지망 같음
    elif (YDS.loc[x][3] == YDS.loc[x][5]) & (YDS.loc[x][3] != '0'):
        C.append(YDS.loc[x].tolist()) # 1,3지망 같음
    elif (YDS.loc[x][4] == YDS.loc[x][5]) & (YDS.loc[x][4] != '0'):
        D.append(YDS.loc[x].tolist()) # 2,3지망 같음
    else:
        E.append(YDS.loc[x].tolist()) # 모두 다름

# 팀 내 금지된 조합 확인 함수
def check_team_conflicts(team):
    # (1) 이동민과 전민지, 한보람, 정윤진은 같은 팀 불가
    if '이동민' in team and any(person in team for person in ['ㅇ', 'ㅇ', 'ㅇ']):
        return True
    # (2) 이선영, 이유진, 유수연은 서로 같은 팀 불가
    if len(set(team).intersection(['ㅇ', 'ㅇ', 'ㅇ'])) > 1:
        return True
    return False

# 팀 배치 후 팀별 금지된 조합을 검사하고, 문제가 있으면 다시 배치
def assign_with_conflict_check(배정리스트, 기업들, 우선지망, 차선지망=None, 세번째지망=None):
    # 각 기업별 최대 인원 수 설정
    max_capacity = {
        'de기업1': 18, 'de기업2': 18, 'de기업3': 18,
        'ds기업1': 18, 'ds기업2': 18, 'ds기업3': 18,
        'deds기업1': 25
    }
    
    for z in 배정리스트:
        key = z[우선지망]  # 우선지망에 따라 배치 시도
        if key in 기업들 and len(기업들[key]) < max_capacity[key]:
            기업들[key].append(z)
            team_members = [person[0] for person in 기업들[key]]
            if check_team_conflicts(team_members):  # 금지된 조합 있는지 확인
                기업들[key].remove(z)  # 금지된 조합이면 팀에서 제외하고 재배치 시도
                if 차선지망:
                    assign_with_conflict_check([z], 기업들, 차선지망, 세번째지망=세번째지망)  # 차선지망으로 재배치
                elif 세번째지망:
                    assign_with_conflict_check([z], 기업들, 세번째지망)  # 3순위 지망으로 배치
        else:
            # 우선지망에 배치 불가능하면 차선지망으로 배치 시도
            if 차선지망 and (z[차선지망] in 기업들 and len(기업들[z[차선지망]]) < max_capacity[z[차선지망]]):
                기업들[z[차선지망]].append(z)
                team_members = [person[0] for person in 기업들[z[차선지망]]]
                if check_team_conflicts(team_members):
                    기업들[z[차선지망]].remove(z)
                    if 세번째지망:
                        assign_with_conflict_check([z], 기업들, 세번째지망)  # 3순위 지망으로 재배치
            elif 세번째지망:
                if len(기업들[z[세번째지망]]) < max_capacity[z[세번째지망]]:
                    기업들[z[세번째지망]].append(z)

# 기존 로직에 각 팀 배치 시 금지된 조합 확인 로직 추가
w = 1
v = 0

while v == 0:
    기업들 = {
        'de기업1': [], 'de기업2': [], 'de기업3': [],
        'ds기업1': [], 'ds기업2': [], 'ds기업3': [],
        'deds기업1': []
    }
    
    # A는 동일 지망자들로 배치
    for z in A:
        key = z[3]
        if key in 기업들:
            기업들[key].append(z)
    
    random.shuffle(B)
    random.shuffle(C)
    random.shuffle(D)
    random.shuffle(E)
    
    # 각 리스트를 배치하면서 팀 조합 체크
    assign_with_conflict_check(B, 기업들, 3, 5)  # B 리스트 1 & 2지망
    assign_with_conflict_check(C, 기업들, 3, 4)  # C 리스트 1 & 3지망
    assign_with_conflict_check(D, 기업들, 3, 4)  # D 리스트 1지망
    assign_with_conflict_check(E, 기업들, 3, 4, 5)  # E 리스트 1지망
    
    # 설문 미제출자 처리
    for z in F:
        if z[2] == 'DE':
            DE_orb = [len(기업들[f'de기업{i+1}']) for i in range(3)] + [len(기업들['deds기업1'])]
            if DE_orb.index(min(DE_orb)) == 3:
                기업들['deds기업1'].append(z)
            else:
                기업들[f'de기업{DE_orb.index(min(DE_orb))+1}'].append(z)
        else:
            DS_orb = [len(기업들[f'ds기업{i+1}']) for i in range(3)] + [len(기업들['deds기업1'])]
            if DS_orb.index(min(DS_orb)) == 3:
                기업들['deds기업1'].append(z)
            else:
                기업들[f'ds기업{DS_orb.index(min(DS_orb))+1}'].append(z)

    # 각 팀의 수준 점수 합계를 확인하는 로직 (기존 코드 유지)
    Topology = []
    for i in range(3):
        u = sum(j[1] for j in 기업들[f'de기업{i+1}'])
        Topology.append(u)

    for i in range(3):
        u = sum(j[1] for j in 기업들[f'ds기업{i+1}'])
        Topology.append(u)
        
    for i in range(1):
        u = sum(j[1] for j in 기업들[f'deds기업{i+1}'])
        Topology.append(u)

    # 합격/불합격 조건 확인 및 재시도 여부
    kbo_1 = sum([row.count('DE') for row in 기업들['deds기업1']])
    kbo_2 = sum([row.count('DS') for row in 기업들['deds기업1']])
    DEDS_orb = [kbo_1, kbo_2]
    
    r = 0
    for i in range(3):
        r += len(기업들[f'de기업{i+1}'])
    for i in range(3):
        r += len(기업들[f'ds기업{i+1}'])
    for i in range(1):
        r += len(기업들[f'deds기업{i+1}'])
    
    if any(36 > x for x in Topology) or any(y_3 < 2 for y_3 in DEDS_orb) or (r != 122):
        w += 1
    else:
        v = 1

# 설계된 1차 팀에서 5 ~ 8명씩 분리 작업하기
팀_목록 = ['de기업1', 'de기업2', 'de기업3', 'ds기업1', 'ds기업2', 'ds기업3', 'deds기업1']
m_목록 = [0] * len(팀_목록)  # m_1, m_2, m_3,...을 리스트로 대체

for i, 팀 in enumerate(팀_목록):
    OMR = len(기업들[팀])
    
    if OMR < 17:
        기업들[f'{팀}_{m_목록[i]}'] = 기업들[팀][:OMR//2]
        m_목록[i] += 1
        기업들[f'{팀}_{m_목록[i]}'] = 기업들[팀][OMR//2:]
    else:
        기업들[f'{팀}_{m_목록[i]}'] = 기업들[팀][:OMR//3]
        m_목록[i] += 1
        기업들[f'{팀}_{m_목록[i]}'] = 기업들[팀][OMR//3:OMR*2//3]
        m_목록[i] += 1
        기업들[f'{팀}_{m_목록[i]}'] = 기업들[팀][OMR*2//3:]
    
    # 최종 팀별 배치 결과 출력
    for k in range(m_목록[i] + 1):
        u = sum(j[1] for j in 기업들[f'{팀}_{k}'])
        team_members = ', '.join([f"({z[0]}, {z[1]})" for z in 기업들[f'{팀}_{k}']])
        print(f'{팀}_{k} : {team_members}')
        print(f'{팀}_{k}의 수준 점수 합 :', u)
        print('\n')

print('작동한 횟수 :', w)

print('전체 인원 =', r)

# 팀별 접두어 정의
팀_접두어 = ['DE_team1_', 'DE_team2_', 'DE_team3_', 'DS_team1_', 'DS_team2_', 'DS_team3_', 'With_team1_']
팀_데이터프레임 = []  # 팀별 데이터프레임을 저장할 리스트

for i, 팀 in enumerate(팀_목록):
    for p in range(m_목록[i] + 1):
        # 팀별로 마지막에 'TEAM' 컬럼을 추가
        기업들[f'{팀}_{p}'] = [sublist + [f'{팀_접두어[i]}{p}'] for sublist in 기업들[f'{팀}_{p}']]
        # DataFrame 생성 및 리스트에 추가
        df = pd.DataFrame(기업들[f'{팀}_{p}'], columns=['이름', '수준', '트랙', '1지망', '2지망', '3지망', '과정', 'TEAM'])
        팀_데이터프레임.append(df)

# 모든 팀의 DataFrame을 concat하여 통합
이어드림_2022 = pd.concat(팀_데이터프레임, ignore_index=True)

# CSV로 저장
이어드림_2022.to_csv("기업연계프로젝트팀을만들어.csv", encoding='utf-8-sig')